In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('/home/heemin/mv/dir/VideoStimSet_SuppTable1.xlsx', engine='openpyxl')
df.head()

/home/heemin/.pyenv/versions/3.6.8/envs/meta/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,video,mode,% correct,Actor,age,gender,disgust (N raters),embarrassment (N raters),fear (N raters),sadness (N raters),happiness (N raters),pride (N raters),neutral (N raters),none (N raters)
0,duckEE10d,sadness,0.611111,10,13,F,8.0,2.0,2.0,22.0,0.0,0.0,0.0,2.0
1,duckEE10e,embarrassment,0.583333,10,13,F,0.0,21.0,0.0,0.0,0.0,2.0,10.0,3.0
2,duckEE10f,fear,0.611111,10,13,F,3.0,4.0,22.0,0.0,0.0,0.0,3.0,4.0
3,duckEE10h,happiness,0.916667,10,13,F,0.0,0.0,0.0,1.0,33.0,0.0,2.0,0.0
4,duckEE10n,neutral,1.000000,10,13,F,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0


In [4]:
df.columns

Index(['video', 'mode', '% correct', 'Actor', 'age', 'gender',
       'disgust (N raters)', 'embarrassment (N raters)', 'fear (N raters)',
       'sadness (N raters)', 'happiness (N raters)', 'pride (N raters)',
       'neutral (N raters)', 'none (N raters)'],
      dtype='object')

In [5]:
df = df[df['% correct']>=0.6]

## data crop

In [6]:
import os
import natsort

from collections import Counter

In [7]:
fl = os.listdir('/home/heemin/mv/dir/DuckEEs')

In [8]:
file_list = natsort.natsorted(fl)

In [9]:
drop_file =[]
for file in file_list:
    df1 = pd.read_csv('/home/heemin/mv/dir/DuckEEs/'+file)[['total_count', 'emotion']]
    if len(df1) == 0:
        drop_file.append(file)

In [10]:
drop_file

['31n.csv', '39p.csv', '39s.csv', '46e.csv', '46h.csv', '46n.csv', '46s.csv']

In [11]:
true_label = df[['video', 'mode']]
true_label['video'] = df['video'].str[6:]
drop_filename = []
for file in drop_file:
    drop_filename.append(file.split('.')[0])
drop_df = pd.DataFrame(drop_filename, columns=['video'])
drop_df['bad'] = 1

/home/heemin/.pyenv/versions/3.6.8/envs/meta/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
label1 = pd.merge(true_label, drop_df, how='left', on='video')
label2 = label1[label1['bad'].isnull()].iloc[:,0:-1]

In [13]:
label2

,video,mode
0,10d,sadness
1,10f,fear
2,10h,happiness
3,10n,neutral
4,10p,pride
...,...,...
173,44s,sadness
174,45h,happiness
175,45n,neutral
176,45p,pride


In [14]:
label2['Neutral'] = 0
label2.loc[label2['mode'] == 'neutral', 'Neutral'] = 1
label2 = label2.reset_index()

In [15]:
label2.to_excel('/home/heemin/mv/dir/DuckEEs_label.xlsx', index=False)

frame to video crop이기 때문에 25frame 마다 crop 할 필요없이 1frame마다 crop하면 됨

In [16]:
for idx in range(len(label2)):
    file = label2['video'][idx]
    df1 = pd.read_csv('/home/heemin/mv/dir/DuckEEs/'+file+'.csv')[['total_count', 'emotion']]
    end_frame = int(df1['total_count'][-1:])
    df2 = pd.merge(pd.DataFrame([i for i in range(1, end_frame+1)], columns=['total_count']), df1, how='outer').fillna('Neutral')
    if label2['Neutral'][idx] == 0:
        crop_frame_start = 0

        for i in range(end_frame):
            if df2['emotion'][i] == 'Neutral' or df2['emotion'][i] == 'none':
                crop_frame_start = i+1
            else:
                break
        crop_frame_end = end_frame

        reverse_df2 = pd.merge(pd.DataFrame([i for i in range(end_frame, -1, -1)], columns=['total_count']), df1, how='outer').fillna('Neutral')
        for i in range(end_frame):
            if reverse_df2['emotion'][i] == 'Neutral' or reverse_df2['emotion'][i] == 'none':
                crop_frame_end = reverse_df2['total_count'][i] - 1
            else:
                break
        if crop_frame_end == 0:
            crop_frame_start = 0
            crop_frame_end = end_frame

        df3 = df2[crop_frame_start:crop_frame_end]
    else:
        df3 = df2
    df3.to_csv('/home/heemin/mv/dir/DuckEEs_crop/'+file+'.csv')

In [17]:
emotion_dict = {'Angry':'Negative', 'Disgust':'Negative', 'Fear':'Negative', 'Happy':'Positive', 
    'Sad':'Negative', 'Suprise':'Negative', 'Neutral':'Neutral'}

In [18]:
duckEE_dict = {'disgust':'Negative', 'fear':'Negative', 'happiness':'Positive',  'embarrassment':'Negative',
    'sadness':'Negative', 'pride':'Positive', 'neutral':'Neutral'}

In [19]:
n_total = 0
n_correct = 0
e_total = 0
e_correct = 0
theshold = 0.275
bad_data = []
for i in range(len(label2)):
    file = label2['video'][i]
    df = pd.read_csv('/home/heemin/mv/dir/DuckEEs_crop/'+file+'.csv', index_col=0)
    label = label2.loc[label2['video'] == file]['mode'][i]

    if label2['Neutral'][i] == 0:
        l1 = df['total_count'].tolist()
        l2 = df['emotion'].tolist()
        df1 = pd.DataFrame([l1,l2]).T
        df1['index'] = df1[0]
        a = list(range(1, l1[-1]+1))
        index_df = pd.DataFrame(a, columns=['index'])
        df2 = pd.merge(index_df, df1, how="outer", on='index')
        df2 = df2.fillna('none')
        emolist = df2[1].to_list()
        emocount = Counter(emolist)
        top3 = emocount.most_common(n=3)
        l3 = []


        total_len = len(l1)
        p10_len = int(total_len * theshold)

        if top3[0][0] == 'none':
            if len(top3) == 1:
                l3.append('Neutral')
            elif top3[1][0] == 'Neutral':
                if len(top3) == 3:
                    if top3[2][1] >= p10_len:
                        l3.append(top3[2][0])
                    else:
                        l3.append(top3[1][0])
                else:
                    l3.append(top3[1][0])
        elif top3[0][0] == 'Neutral':
            if len(top3) == 1:
                l3.append('Neutral')
            elif top3[1][0] == 'none':
                if len(top3) == 2: 
                    l3.append('Neutral')
                else:
                    if top3[2][1] >= p10_len:
                        l3.append(top3[2][0])
                    else:
                        l3.append(top3[0][0])
            else:
                if top3[1][1] >= p10_len:
                    l3.append(top3[1][0])
                else:
                    l3.append(top3[0][0])
        else:
            l3.append(top3[0][0])
        for emotion in l3:
            e_total+=1
            if emotion_dict[emotion] == duckEE_dict[label]:
                e_correct += 1
            else:
                bad_data.append([file, emotion, label])
            print(file, emotion, label, emotion_dict[emotion], duckEE_dict[label])
    
    else:
        l1 = df['total_count'].tolist()
        l2 = df['emotion'].tolist()
        df1 = pd.DataFrame([l1,l2]).T
        df1['index'] = df1[0]
        a = list(range(1, l1[-1]+1))
        index_df = pd.DataFrame(a, columns=['index'])
        df2 = pd.merge(index_df, df1, how="outer")
        df2 = df2.fillna('none')
        emolist = df2[1].to_list()
        emocount = Counter(emolist)
        top3 = emocount.most_common(n=3)
        l3 = []
        
        if top3[0][0] == 'none':
            l3.append('Neutral')
        else:
            l3.append(top3[0][0])

        for emotion in l3:
            n_total+=1
            if emotion_dict[emotion] == duckEE_dict[label]:
                n_correct += 1
            else:
                bad_data.append([file, emotion, label])
            print(file, emotion, label, emotion_dict[emotion], duckEE_dict[label])
    
n_acc = n_correct / n_total
e_acc = e_correct / e_total
acc = (n_correct+e_correct) / (n_total+e_total)
print(f'Accuracy for Neutral data: {n_acc*100:0.2f}%\nAccuracy for Non-Neutral data: {e_acc*100:0.2f}%\nAccuracy for Total data: {acc*100:0.2f}%\n')

10d Sad sadness Negative Negative
10h Happy happiness Positive Positive
10n Sad neutral Negative Neutral
10p Happy pride Positive Positive
10s Sad sadness Negative Negative
11e Happy embarrassment Positive Negative
11f Suprise fear Negative Negative
11h Happy happiness Positive Positive
11n Neutral neutral Neutral Neutral
11p Happy pride Positive Positive
12f Angry fear Negative Negative
12h Happy happiness Positive Positive
12n Neutral neutral Neutral Neutral
13d Sad disgust Negative Negative
13e Happy embarrassment Positive Negative
13f Sad fear Negative Negative
13h Happy happiness Positive Positive
13p Happy pride Positive Positive
14d Disgust disgust Negative Negative
14h Happy happiness Positive Positive
15n Suprise neutral Negative Neutral
16e Sad embarrassment Negative Negative
16f Neutral fear Neutral Negative
16h Happy happiness Positive Positive
16n Neutral neutral Neutral Neutral
16p Neutral pride Neutral Positive
17d Angry disgust Negative Negative
17h Happy happiness Posi

In [20]:
top3

[('Angry', 15), ('none', 9), ('Sad', 8)]

In [21]:
j = pd.DataFrame(bad_data, columns=['file', 'emotion', 'label'])

In [22]:
# pd.DataFrame(bad_data, columns=['file', 'emotion', 'label']).to_excel('/Users/heemin/Documents/DHLab/mv/project/baddata.xlsx')

In [23]:
n_total = 0
n_correct = 0
e_total = 0
e_correct = 0
theshold = 0.275
bad_data = []
for i in range(len(label2)):
    file = label2['video'][i]
    df = pd.read_csv('/home/heemin/mv/dir/DuckEEs_crop/'+file+'.csv', index_col=0)
    label = label2.loc[label2['video'] == file]['mode'][i]

    if label2['Neutral'][i] == 0:
        l1 = df['total_count'].tolist()
        l2 = df['emotion'].tolist()
        df1 = pd.DataFrame([l1,l2]).T
        df1['index'] = df1[0]
        a = list(range(l1[-1]+1))
        index_df = pd.DataFrame(a, columns=['index'])
        df2 = pd.merge(index_df, df1, how="outer")
        df2 = df2.fillna('none')
        emolist = df2[1].to_list()
        emocount = Counter(emolist)
        top3 = emocount.most_common(n=3)
        l3 = []


        total_len = len(l1)
        p10_len = int(total_len * theshold)

        if top3[0][0] == 'none':
            if len(top3) == 1:
                l3.append('Neutral')
            elif top3[1][0] == 'Neutral':
                if len(top3) == 3:
                    if top3[2][1] >= p10_len:
                        l3.append(top3[2][0])
                    else:
                        l3.append(top3[1][0])
                else:
                    l3.append(top3[1][0])
        elif top3[0][0] == 'Neutral':
            if len(top3) == 1:
                l3.append('Neutral')
            elif top3[1][0] == 'none':
                if len(top3) == 2: 
                    l3.append('Neutral')
                else:
                    if top3[2][1] >= p10_len:
                        l3.append(top3[2][0])
                    else:
                        l3.append(top3[0][0])
            else:
                if top3[1][1] >= p10_len:
                    l3.append(top3[1][0])
                else:
                    l3.append(top3[0][0])
        else:
            l3.append(top3[0][0])
        for emotion in l3:
            e_total+=1
            if emotion_dict[emotion] == duckEE_dict[label]:
                e_correct += 1
            else:
                bad_data.append([file, emotion, label])
            print(file, emotion, label, emotion_dict[emotion], duckEE_dict[label])
    
    else:
        l1 = df['total_count'].tolist()
        l2 = df['emotion'].tolist()
        df1 = pd.DataFrame([l1,l2]).T
        df1['index'] = df1[0]
        a = list(range(l1[-1]+1))
        index_df = pd.DataFrame(a, columns=['index'])
        df2 = pd.merge(index_df, df1, how="outer")
        df2 = df2.fillna('none')
        emolist = df2[1].to_list()
        emocount = Counter(emolist)
        top3 = emocount.most_common(n=3)
        l3 = []


        total_len = len(l1)
        p10_len = int(total_len * theshold)

        if top3[0][0] == 'none':
            if len(top3) == 1:
                l3.append('Neutral')
            elif top3[1][0] == 'Neutral':
                if len(top3) == 3:
                    if top3[2][1] >= p10_len:
                        l3.append(top3[2][0])
                    else:
                        l3.append(top3[1][0])
                else:
                    l3.append(top3[1][0])
        elif top3[0][0] == 'Neutral':
            if len(top3) == 1:
                l3.append('Neutral')
            elif top3[1][0] == 'none':
                if len(top3) == 2: 
                    l3.append('Neutral')
                else:
                    if top3[2][1] >= p10_len:
                        l3.append(top3[2][0])
                    else:
                        l3.append(top3[0][0])
            else:
                if top3[1][1] >= p10_len:
                    l3.append(top3[1][0])
                else:
                    l3.append(top3[0][0])
        else:
            l3.append(top3[0][0])
        for emotion in l3:
            n_total+=1
            if emotion_dict[emotion] == duckEE_dict[label]:
                n_correct += 1
            else:
                bad_data.append([file, emotion, label])
            print(file, emotion, label, emotion_dict[emotion], duckEE_dict[label])
    
n_acc = n_correct / n_total
e_acc = e_correct / e_total
acc = (n_correct+e_correct) / (n_total+e_total)
print(f'Accuracy for Neutral data: {n_acc*100:0.2f}%\nAccuracy for Non-Neutral data: {e_acc*100:0.2f}%\nAccuracy for Total data: {acc*100:0.2f}%\n')

10d Sad sadness Negative Negative
10h Happy happiness Positive Positive
10n Sad neutral Negative Neutral
10p Happy pride Positive Positive
10s Sad sadness Negative Negative
11e Happy embarrassment Positive Negative
11f Suprise fear Negative Negative
11h Happy happiness Positive Positive
11n Neutral neutral Neutral Neutral
11p Happy pride Positive Positive
12f Angry fear Negative Negative
12h Happy happiness Positive Positive
12n Neutral neutral Neutral Neutral
13d Sad disgust Negative Negative
13e Happy embarrassment Positive Negative
13f Sad fear Negative Negative
13h Happy happiness Positive Positive
13p Happy pride Positive Positive
14d Disgust disgust Negative Negative
14h Happy happiness Positive Positive
15n Suprise neutral Negative Neutral
16e Sad embarrassment Negative Negative
16f Neutral fear Neutral Negative
16h Happy happiness Positive Positive
16n Neutral neutral Neutral Neutral
16p Neutral pride Neutral Positive
17d Angry disgust Negative Negative
17h Happy happiness Posi

In [59]:
k = pd.DataFrame(bad_data, columns=['file', 'emotion', 'label'])

In [61]:
k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   file     24 non-null     object
 1   emotion  24 non-null     object
 2   label    24 non-null     object
dtypes: object(3)
memory usage: 704.0+ bytes


In [62]:
j.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   file     31 non-null     object
 1   emotion  31 non-null     object
 2   label    31 non-null     object
dtypes: object(3)
memory usage: 872.0+ bytes


In [63]:
pd.merge(j, k, how='left', on='file')

,file,emotion_x,label_x,emotion_y,label_y
0,10n,Sad,neutral,NaN,NaN
1,11e,Happy,embarrassment,Happy,embarrassment
2,13e,Happy,embarrassment,Happy,embarrassment
3,15n,Suprise,neutral,NaN,NaN
4,16f,Neutral,fear,Neutral,fear
5,16p,Neutral,pride,Neutral,pride
6,18e,Neutral,embarrassment,Neutral,embarrassment
7,18n,Fear,neutral,NaN,NaN
8,18s,Sad,neutral,NaN,NaN
9,20n,Angry,neutral,NaN,NaN


In [23]:
bd = pd.DataFrame(bad_data, columns=['file', 'emotion', 'label'])
bd[bd['label']=='Positive'][['emotion', 'label']].groupby('emotion').count().sort_values(by='label', ascending=False).reset_index()

,emotion,label


In [ ]:
bd[bd['label']=='Negative'][['emotion', 'label']].groupby('emotion').count().sort_values(by='label', ascending=False).reset_index()

,emotion,label
0,Neutral,2
1,Happy,1


In [ ]:
bd[bd['label']=='Neutral'][['emotion', 'label']].groupby('emotion').count().sort_values(by='label', ascending=False).reset_index()

,emotion,label
0,Fear,2
1,Sad,2
2,Angry,1


In [ ]:
# bd[bd['label']=='Neutral'].to_excel('/Users/heemin/Documents/DHLab/mv/project/baddata_Neutral.xlsx')

In [74]:
n_total = 0
n_correct = 0
e_total = 0
e_correct = 0
theshold = 0.275
bad_data = []
for i in range(len(label2)):
    file = label2['video'][i]
    df = pd.read_csv('/home/heemin/mv/dir/DuckEEs_crop/'+file+'.csv', index_col=0)
    label = label2.loc[label2['video'] == file]['mode'][i]

    if label2['Neutral'][i] == 0:
        l1 = df['total_count'].tolist()
        l2 = df['emotion'].tolist()
        df1 = pd.DataFrame([l1,l2]).T
        df1['index'] = df1[0]
        a = list(range(1, l1[-1]+1))
        index_df = pd.DataFrame(a, columns=['index'])
        df2 = pd.merge(index_df, df1, how="outer")
        df2 = df2.fillna('none')
        emolist = df2[1].to_list()
        emocount = Counter(emolist)
        top3 = emocount.most_common(n=3)
        l3 = []
        
        if top3[0][0] == 'none':
            l3.append('Neutral')
        else:
            l3.append(top3[0][0])
        for emotion in l3:
            e_total+=1
            if emotion_dict[emotion] == duckEE_dict[label]:
                e_correct += 1
            else:
                bad_data.append([file, emotion, label])
            print(file, emotion, label, emotion_dict[emotion], duckEE_dict[label])
    
    else:
        l1 = df['total_count'].tolist()
        l2 = df['emotion'].tolist()
        df1 = pd.DataFrame([l1,l2]).T
        df1['index'] = df1[0]
        a = list(range(1, l1[-1]+1))
        index_df = pd.DataFrame(a, columns=['index'])
        df2 = pd.merge(index_df, df1, how="outer")
        df2 = df2.fillna('none')
        emolist = df2[1].to_list()
        emocount = Counter(emolist)
        top3 = emocount.most_common(n=3)
        l3 = []
        
        if top3[0][0] == 'none':
            l3.append('Neutral')
        else:
            l3.append(top3[0][0])

        for emotion in l3:
            n_total+=1
            if emotion_dict[emotion] == duckEE_dict[label]:
                n_correct += 1
            else:
                bad_data.append([file, emotion, label])
            print(file, emotion, label, emotion_dict[emotion], duckEE_dict[label])
    
n_acc = n_correct / n_total
e_acc = e_correct / e_total
acc = (n_correct+e_correct) / (n_total+e_total)
print(f'Accuracy for Neutral data: {n_acc*100:0.2f}%\nAccuracy for Non-Neutral data: {e_acc*100:0.2f}%\nAccuracy for Total data: {acc*100:0.2f}%\n')

10d Sad sadness Negative Negative
10f Neutral fear Neutral Negative
10h Happy happiness Positive Positive
10n Sad neutral Negative Neutral
10p Happy pride Positive Positive
10s Sad sadness Negative Negative
11e Happy embarrassment Positive Negative
11f Suprise fear Negative Negative
11h Happy happiness Positive Positive
11n Neutral neutral Neutral Neutral
11p Happy pride Positive Positive
11s Neutral sadness Neutral Negative
12f Angry fear Negative Negative
12h Happy happiness Positive Positive
12n Neutral neutral Neutral Neutral
13d Sad disgust Negative Negative
13e Happy embarrassment Positive Negative
13f Sad fear Negative Negative
13h Happy happiness Positive Positive
13p Happy pride Positive Positive
14d Disgust disgust Negative Negative
14h Neutral happiness Neutral Positive
15n Suprise neutral Negative Neutral
16e Sad embarrassment Negative Negative
16f Neutral fear Neutral Negative
16h Happy happiness Positive Positive
16n Neutral neutral Neutral Neutral
16p Neutral pride Neutr